In [1]:
import pandas as pd

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [5]:
from collections import namedtuple

In [6]:
DataIngenstionConfig = namedtuple("DataIngenstionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngenstionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir: str


In [7]:
from imp import source_from_cache


class ConfigManager:
    def __init__(self) -> None:
        config_path = CONFIG_FILE_PATH
        params_path = PARAMS_FILE_PATH
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngenstionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingention_config = DataIngenstionConfig( 
                root_dir= config.root_dir ,
                source_URL=  config.source_URL,
                local_data_file= config.local_data_file,
                unzip_dir= config.unzip_dir

            )

        return data_ingention_config

/tmp/ipykernel_64149/1204788099.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import source_from_cache


In [8]:
print(ConfigManager().get_data_ingestion_config())

[2022-09-13 15:02:49,486: INFO: common: yaml file: configs/config.yaml loaded successfully]
[2022-09-13 15:02:49,487: INFO: common: yaml file: parameter.yaml loaded successfully]
[2022-09-13 15:02:49,488: INFO: common: created directory at: artifacts]
[2022-09-13 15:02:49,489: INFO: common: created directory at: artifacts/data_ingestion]
DataIngenstionConfig(root_dir='artifacts/data_ingestion', source_URL='https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')


In [9]:
import os
import urllib.request as request
from zipfile import ZipFile


In [10]:
class DataIngestion:
    def __init__(self, config:DataIngenstionConfig) -> None:
        self.config = config

    def download_file(self):
        file_name = self.config.local_data_file

        if not os.path.exists(file_name):
            result_file_name, header = request.urlretrieve(url= self.config.source_URL,filename=file_name)
    
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)
            
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [11]:
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2022-09-13 15:02:49,612: INFO: common: yaml file: configs/config.yaml loaded successfully]
[2022-09-13 15:02:49,613: INFO: common: yaml file: parameter.yaml loaded successfully]
[2022-09-13 15:02:49,614: INFO: common: created directory at: artifacts]
[2022-09-13 15:02:49,614: INFO: common: created directory at: artifacts/data_ingestion]
